In [67]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import defaultdict
from IPython.display import display
from fuzzywuzzy import fuzz, process
from ITUtils import country_conflicts_finder
from pandarallel import pandarallel

# show all columns
pd.set_option('display.max_rows', None)

# Initialize pandarallel
num_logical_processors = os.cpu_count()  # if run on a cluster change this manually
pandarallel.initialize(nb_workers=num_logical_processors, progress_bar=True)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [68]:
# pick a country and import the conflicts
adm = 'CAN'
file = 'expanded_combined_tables_conflicts_othersatnames.csv'
filepath = os.path.join('.', 'adm_conflicts', adm, file)
df = pd.read_csv(filepath, low_memory=False)
display(df.head())
print(df.columns)

,com_el.ntc_id,com_el.tgt_ntc_id,com_el.adm,com_el.ntwk_org,com_el.sat_name,com_el.prov,com_el.d_rcv,com_el.st_cur,orbit.orb_id,orbit.nbr_sat_pl,...,TPA1.2055.475-2055.725_R,TPA1.401.89525-401.90475_R,TPA1.2237.0-2238.0_E,TPA1.2065.575-2065.825_R,TPA1.401.9501-401.9699_E,TPA1.401.8901-401.9099_R,TPA1.401.95525-401.96475_E,TPA1.401.8901-401.9099_E,TPA1.401.89525-401.90475_E,TPA1.2202.4-2203.4_E
0,117545422,NaN,CAN,NaN,MULTUS,NaN,9.1/IA,4/12/2017 12:00:00 am,50,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,117545422,NaN,CAN,NaN,MULTUS,NaN,9.1/IA,4/12/2017 12:00:00 am,50,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,117545422,NaN,CAN,NaN,MULTUS,NaN,9.1/IA,4/12/2017 12:00:00 am,50,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,117545422,NaN,CAN,NaN,MULTUS,NaN,9.1/IA,4/12/2017 12:00:00 am,50,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,117545422,NaN,CAN,NaN,MULTUS,NaN,9.1/IA,4/12/2017 12:00:00 am,50,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Index(['com_el.ntc_id', ' com_el.tgt_ntc_id', ' com_el.adm',
       ' com_el.ntwk_org', ' com_el.sat_name', ' com_el.prov', ' com_el.d_rcv',
       ' com_el.st_cur', ' orbit.orb_id', ' orbit.nbr_sat_pl',
       ' orbit.inclin_ang', ' orbit.apog_km', ' orbit.perig_km',
       ' orbit.op_ht_km', ' s_beam.emi_rcp', ' s_beam.beam_name',
       ' grp.grp_id', ' grp.freq_min', ' grp.freq_max', ' grp.bdwdth',
       ' grp.d_inuse', ' grp.d_reg_limit', ' grp.d_prot_eff', ' grp.f_biu',
       ' emiss.seq_no', ' emiss.pwr_ds_max', ' emiss.design_emi',
       ' carrier_fr.freq_carr', ' carrier_fr.seq_emiss', ' carrier_fr.seq_no',
       ' srv_area.ctry', ' srv_area.f_excl_api', ' channel.bandwidth',
       ' channel.freq_min', ' channel.freq_max', 'tpaconflicts',
       'percentoverlap', 'TPA1.401.95525-401.96475_R',
       'TPA1.401.9501-401.9699_R', 'TPA1.2055.475-2055.725_R',
       'TPA1.401.89525-401.90475_R', 'TPA1.2237.0-2238.0_E',
       'TPA1.2065.575-2065.825_R', 'TPA1.401.9501-401.9699

In [69]:
# function to extract the table

def condensed_summary_table(df, lic):
    # Columns to keep
    cols = [
        'com_el.ntc_id',  # todo check if it is contained in the RSM list
        ' com_el.sat_name',
        ' s_beam.beam_name',
        ' carrier_fr.freq_carr',
        ' channel.bandwidth',
        'tpaconflicts',
        'percentoverlap',
        ' srv_area.ctry'
    ]
    conflict_areas = [  # for the definitions see table 1B from preface
        'XR3',  # ITU Region 3 (Oceania)
        'NZL',  # New Zealand
        'XAA',  # World
        'XAX',  # unspecified general restricted area with multiple countries
        'XUN',  # united nations
        'XVE',  # visible earth
        'XMM',  # inter-ship communications
        'XOA',  # international Civil Aviation Organisation
        'XWM'  # World metereological organisation  
    ]
    lookupnames = [
        'UHFUP fc=401.96MHz BW=9.5kHz',
        'UHFUP fc=401.96MHz BW=19.8kHz',
        'SUP fc=2055.6MHz BW=250kHz',
        'UHFUP fc=401.90MHz BW=9.5kHz',
        'SDN fc=2237.5MHz BW=1MHz',
        'SUP fc=2065.7MHz BW=250kHz',
        'UHFDN fc=401.96MHz BW=19.8kHz',
        'UHFUP fc=401.90MHz BW=19.8kHz',
        'UHFDN fc=401.96MHz BW=9.5kHz',
        'UHFDN fc=401.90MHz BW=19.8kHz',
        'UHFDN fc=401.90MHz BW=9.5kHz',
        'SDN fc=2202.9MHz BW=1MHz'
    ]

    df1 = df[cols].copy()
    if df1.empty:
        print("❗ DataFrame is empty")
        data = {
            'Network': '',
            'Beam': 'All',
            'TPA-1 Beam': 'No Conflict',
            'srv area': '',
            'RSMlic': '',
            'Overlap (worst case)':''
        }
        return pd.DataFrame([data])

    def functiontoapply(row):

        import pandas as pd  # otherwhise it doesn't work under windows 

        satname = row[' com_el.sat_name'].strip()
        beamname = row[' s_beam.beam_name'].strip()
        full_id = f"{satname} - {beamname}"

        # extract the conflict indicese and percentuals
        con = str(row['tpaconflicts']).split(':')
        percent_str = str(row['percentoverlap']).split(':')
        conflict_idxs = []
        for c in con[:-1]:
            conflict_idxs.append(int(c))
        percents = str(row['percentoverlap']).split(':')

        # no conflicts case, I still need to return the satellite name (duplicates can be removed at the end) 
        # This is also the table prototype
        if not conflict_idxs or not percents:
            srv_areas = df.loc[df[' com_el.sat_name'] == satname, ' srv_area.ctry'].unique()
            srv_area = srv_areas.astype(str).tolist()
            srv_area = [str(item) for item in srv_area if item in conflict_areas]
            srv_area = ':'.join(srv_area) if srv_area else ''
            data = {
                'Network': str(satname),
                'Beam': 'All',
                'TPA-1 Beam': 'No Conflict',
                'percent': 0.0,  # within this function this needs to be just a number
                'fc': 0.0,  # needed for the expanded conflict literal
                'bw': 0.0,  # needed fot the expanded conflict literal
                'srv area': srv_area,
                'RSMlic': 'Y' if str(row['com_el.ntc_id']) in lic else 'N'
            }
            return pd.DataFrame([data])

        min_len = min(len(conflict_idxs), len(percents))
        fc = float(row[' carrier_fr.freq_carr'])
        bw = float(row[' channel.bandwidth'])  # Hz
        data = [
            {
                'Network': str(satname),
                'Beam': str(beamname),
                'TPA-1 Beam': lookupnames[conflict_idxs[i]],
                'percent': float(percents[i]) if percents[i] else 0.0,
                'fc': fc,
                'bw': bw,
                'srv area': row[' srv_area.ctry'] if row[' srv_area.ctry'] in conflict_areas else '',
                'RSMlic': 'Y' if str(row['com_el.ntc_id']) in lic else 'N'
            }
            for i in range(min_len)
            if 0 <= conflict_idxs[i] < len(lookupnames)
        ]

        return pd.DataFrame(data)

    # Apply in parallel or serial depending on size
    if len(df1) > 1000:
        print("parallelising functiontoapply in condensed_summary_table")
        expanded_rows = df1.parallel_apply(functiontoapply, axis=1)
    else:
        tqdm.pandas()
        expanded_rows = df1.apply(functiontoapply, axis=1)
    # tqdm.pandas()
    # expanded_rows = df1.apply(functiontoapply, axis=1)

    # Flatten the list of DataFrames
    df_expanded = pd.concat(expanded_rows.tolist(), ignore_index=True)

    if df_expanded.empty:
        print("⚠️ No valid conflict data found.")
        data = {
            'Network': '',
            'Beam': 'All',
            'TPA-1 Beam': 'No Conflict',
            'srv area': '',
            'RSMlic': '',
            'Overlap (worst case)': ''
        }
        return pd.DataFrame([data])

    # Get the row with max percent per group
    df_expanded.sort_values('percent', ascending=False, inplace=True)  # sort by percent
    grouped = df_expanded.groupby(['Network', 'Beam', 'TPA-1 Beam'],
                                  as_index=False).first()  # isolate the beam--TPAbeam conflicts

    # addd the literal Overlap stringdef format_overlap_string(row):

    def format_overlap_string(row):
        if row['fc'] == 0.0:
            row['TPA-1 Beam'] = 'No Info'
            return 'No data on fc'

        if row['TPA-1 Beam'] == 'No Conflict':
            return ''

        if row.bw >= 1e6:
            bw_str = f"{row.bw / 1e6:.3f}MHz"
        else:
            bw_str = f"{row.bw / 1e3:.3f}kHz"
        return f"{row.percent}% @ fc={row.fc:.3f}MHz, BW={bw_str}"

    if len(grouped) > 100:
        print("parallelising format_overlap_string")
        grouped['Overlap (worst case)'] = grouped.parallel_apply(format_overlap_string, axis=1)
    else:
        grouped['Overlap (worst case)'] = grouped.apply(format_overlap_string, axis=1)

    def correctnoinfo(row):
        if row['Overlap (worst case)'] == 'No data on fc':
            return 'No Info'
        else:
            return row['TPA-1 Beam']

    if len(grouped) > 100:
        print("parallelising correctnoinfo")
        grouped['TPA-1 Beam'] = grouped.parallel_apply(correctnoinfo, axis=1)
    else:
        grouped['TPA-1 Beam'] = grouped.apply(correctnoinfo, axis=1)

    # Drop the columns 'bw', 'fc', and 'percent'
    grouped = grouped.drop(columns=['bw', 'fc', 'percent'])

    # Drop all redundant lines
    grouped = grouped.drop_duplicates()

    # Make a hard copy of the DataFrame
    grouped_copy = grouped.copy()

    # Return the hard copy of the DataFrame
    return grouped_copy



In [70]:
# function to append satellite names without info in the ITU database
def noinfo_appender(df, satnamesfolder, countrycode, conflictdatabase=None):
    """

    :param df: summary table database
    :param satnamesfolder:
    :param countrycode:
    :return:
    """
    filename = os.path.join('.', satnamesfolder, f'{countrycode}.txt')
    # read file rows as a list
    satnames = []
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            line = f.read()
            satnames = [name.strip() for name in line.split(',')]

    # get all satellite names
    if not df.empty:
        satfound = df['Network'].unique().tolist()
    else:
        satfound = []
    satnotfound = [name for name in satnames if name not in satfound]

    # if a conflict database is provided then use that to find satellite names
    if conflictdatabase is not None:
        dbsatnames = conflictdatabase[' com_el.sat_name'].unique()
        newsatnames = []
        for s in dbsatnames:
            if s.strip() not in satnames:
                newsatnames.append(s)
        satnotfound = newsatnames
    print(satnotfound)
    # append to dataframe
    rows_to_append = []
    for sat in satnotfound:
        srv_areas = df.loc[df['Network'] == sat, 'srv area'].tolist()
        srv_areas = list(set(srv_areas))  # unique
        srv_areas = ':'.join(srv_areas)  # to string
        data = {
            'Network': sat,
            'Beam': 'All',
            'TPA-1 Beam': 'No Info',
            'Overlap (worst case)': 'No data on fc',
            'srv area': srv_areas,
            'RSMlic': ''
        }
        rows_to_append.append(data)

    # Append all missing rows at once
    if rows_to_append:
        df = pd.concat([df, pd.DataFrame(rows_to_append)], ignore_index=True)

    return df.copy()

In [77]:
# usage

with open('RSM_licences.csv', 'r') as file:
    content = file.read()
    licences = [item.strip() for item in content.split(',')]
print(type(licences[0]))

<class 'str'>


In [72]:
pivot_df = condensed_summary_table(df, licences)
pivot_df = noinfo_appender(pivot_df, 'satellitenames', adm)
pivot_df.to_csv('conflict_percent_pivot.csv')
display(pivot_df)
# othersantnames


['ADS', 'BRITE-CANADA', 'CANSAT-LEO-S2', 'CANX-2', 'CANX-4-5', 'CASSIOPE-1A', 'CHORUS-C1', 'EV10', 'GHGSAT-CX', 'GHGSAT-D', 'GJP', 'HOTH', 'KELYPSIS', 'M3MSAT', 'NEOSSAT-1A', 'QEYSSAT', 'RADARSAT-2C', 'RADARSAT-3C', 'SAPPHIRE', 'SCISAT-1']


,Network,Beam,TPA-1 Beam,srv area,RSMlic,Overlap (worst case)
0,MOBIUS-1,All,No Info,XAA,N,No data on fc
1,MULTUS,All,No Info,XAA,N,No data on fc
2,MULTUS,TTCDL,UHFDN fc=401.96MHz BW=19.8kHz,XAA,N,"49.6% @ fc=401.950MHz, BW=25.000kHz"
3,MULTUS,TTCDL,UHFDN fc=401.96MHz BW=9.5kHz,XAA,N,"29.0% @ fc=401.950MHz, BW=25.000kHz"
4,MULTUS,TTCDL,UHFUP fc=401.96MHz BW=19.8kHz,XAA,N,"49.6% @ fc=401.950MHz, BW=25.000kHz"
5,MULTUS,TTCDL,UHFUP fc=401.96MHz BW=9.5kHz,XAA,N,"29.0% @ fc=401.950MHz, BW=25.000kHz"
6,NORTHSTAR2,All,No Info,,N,No data on fc
7,SEDNA,All,No Info,,N,No data on fc
8,SEDNA,SDL,SDN fc=2202.9MHz BW=1MHz,,N,"60.0% @ fc=2202.500MHz, BW=1.000MHz"
9,ADS,All,No Info,,,No data on fc


In [73]:
import os

# === CONFIG ===
tpafile = './databases/TPAtable.csv'
tablesfolder = 'countriestables'
outfolder = 'adm_conflicts'
countrieslistfile = 'countrieslist.csv'

# Load country codes
with open(countrieslistfile, 'r') as f:
    countries = f.read().strip().split(', ')
# # todo comment this
countries = ['CAN','HOL']

# === PROCESS EACH COUNTRY ===
for ccode in countries:
    print(f"\n=== Processing {ccode} ===")

    # outfolder (must already exist)
    country_outfolder = os.path.join(outfolder, ccode)

    # Read data for the 'expanded_combined_tables_conflicts_lettersatnames.csv' file
    adm = ccode
    file = 'expanded_combined_tables_conflicts_lettersatnames.csv'
    filepath = os.path.join('.', 'adm_conflicts', adm, file)
    df = pd.read_csv(filepath, low_memory=False)

    # Generate summary pivot table
    # print(df)
    summary_pivot = condensed_summary_table(df, licences)
    # append noinfo
    # print(summary_pivot)
    summary_pivot = noinfo_appender(summary_pivot, 'satellitenames', ccode)
    # save
    outpath = os.path.join(country_outfolder, 'conflicts_summary_condensed.csv')
    summary_pivot.to_csv(outpath, index=False)
    print('Summary condensed saved to ', outpath)

    # Read data for the 'expanded_combined_tables_conflicts_othersatnames.csv' file
    file = 'expanded_combined_tables_conflicts_othersatnames.csv'
    filepath = os.path.join('.', 'adm_conflicts', adm, file)
    df = pd.read_csv(filepath, low_memory=False)

    # Generate summary pivot table for other satellite names
    summary_pivot_othersatnames = condensed_summary_table(df, licences)
    outpath = os.path.join(country_outfolder, 'conflicts_summary_condensed_othersatnames.csv')
    summary_pivot_othersatnames.to_csv(outpath, index=False)

    print('Summary condensed for other satellite names saved to ', outpath)

    # from IPython.display import clear_output
    # clear_output(wait=True)





=== Processing CAN ===
parallelising functiontoapply in condensed_summary_table


[]
Summary condensed saved to  adm_conflicts\CAN\conflicts_summary_condensed.csv
Summary condensed for other satellite names saved to  adm_conflicts\CAN\conflicts_summary_condensed_othersatnames.csv

=== Processing HOL ===
❗ DataFrame is empty
[]
Summary condensed saved to  adm_conflicts\HOL\conflicts_summary_condensed.csv
Summary condensed for other satellite names saved to  adm_conflicts\HOL\conflicts_summary_condensed_othersatnames.csv
